In [ ]:
ver = 'v7'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import gc
from tqdm import *

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('sample_submission.csv')

In [ ]:
df = pd.concat([train, test]).reset_index(drop = True)

In [ ]:
df.nunique()

Area_Code        30900
Locality_Code       22
Region_Code       2185
Height             110
Diameter           151
Class                8
Species            263
dtype: int64

In [ ]:
df['Region_Species'] = pd.factorize(df['Region_Code'].astype('str') + df['Species'].astype('str'))[0]
df['Region_Species'].nunique()

17667

In [ ]:
df['height*diameter'] = df['Height'] * df['Diameter']
df['height/diameter'] = df['Height'] / df['Diameter']
df['log_diameter'] = np.log(df['Diameter'])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
cat_fts = ['Area_Code', 'Locality_Code', 'Region_Code', 'Species', 'Region_Species']
num_fts = ['Height', 'Diameter','height*diameter', 'height/diameter', 'log_diameter']

for g in tqdm_notebook(cat_fts):

  grp = df.groupby(g)[num_fts].mean()
  grp.columns = [c + f'_grpd_by_{g}_mean' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].std()
  grp.columns = [c + f'_grpd_by_{g}_std' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].min()
  grp.columns = [c + f'_grpd_by_{g}_min' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].max()
  grp.columns = [c + f'_grpd_by_{g}_max' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
fts = ['Area_Code','Locality_Code','Region_Code','Height','Diameter', 'Species', 'Region_Species']
for g in tqdm_notebook(fts):
  rf = [c for c in fts if c != g]
  grp = df.groupby(g)[rf].nunique()
  grp.columns = [c + f'_grpd_by_{g}_nunique' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
for f in fts:
  df[f + '_freq'] = df[f].map(df[f].value_counts(normalize = True))

In [ ]:
train, test = df[:train.shape[0]].reset_index(drop = True), df[train.shape[0]:].reset_index(drop = True)

In [ ]:
features = [c for c in train.columns if c not in ['Class', "Area_Code", "Region_Code", 'Region_Species']]

In [ ]:
len(features)

156

In [ ]:
oofs = np.zeros((len(train), 8))
preds = np.zeros((len(test), 8))
N_SPLITS = 15
folds = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 77)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['Class'])):

  print(f'\n\n Fold {fold_} \\n\n')
  X_trn, y_trn = train[features].iloc[trn_idx], train["Class"][trn_idx]
  X_val, y_val = train[features].iloc[val_idx], train["Class"][val_idx]

  clf = XGBClassifier(n_estimators = 2000, learning_rate = 0.03, colsample_bytree = 0.6, colsample_bynode=0.6, reg_alpha = 0.5, reg_lambda = 4, random_state = 2,
                      max_depth = 16, tree_method = 'gpu_hist')

  clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 200, verbose = 50, eval_metric = 'mlogloss')

  oofs[val_idx] = clf.predict_proba(X_val)
  preds += clf.predict_proba(test[features]) / N_SPLITS




 Fold 0 \n

[0]	validation_0-mlogloss:2.0218
Will train until validation_0-mlogloss hasn't improved in 200 rounds.
[50]	validation_0-mlogloss:1.0492
[100]	validation_0-mlogloss:0.846962
[150]	validation_0-mlogloss:0.784406
[200]	validation_0-mlogloss:0.76648
[250]	validation_0-mlogloss:0.762545
[300]	validation_0-mlogloss:0.766154
[350]	validation_0-mlogloss:0.77301
[400]	validation_0-mlogloss:0.780924
[450]	validation_0-mlogloss:0.787922
Stopping. Best iteration:
[251]	validation_0-mlogloss:0.762499



 Fold 1 \n

[0]	validation_0-mlogloss:2.02554
Will train until validation_0-mlogloss hasn't improved in 200 rounds.
[50]	validation_0-mlogloss:1.03854
[100]	validation_0-mlogloss:0.828675
[150]	validation_0-mlogloss:0.765145
[200]	validation_0-mlogloss:0.748275
[250]	validation_0-mlogloss:0.745883
[300]	validation_0-mlogloss:0.750678
[350]	validation_0-mlogloss:0.757734
[400]	validation_0-mlogloss:0.766202
[450]	validation_0-mlogloss:0.775004
Stopping. Best iteration:
[250]	validation

In [ ]:
oof_val_score = log_loss(train['Class'], oofs)
print(f'Final Log loss: {oof_val_score}')

Final Log loss: 0.7380900319264871


In [ ]:
ss[ss.columns.tolist()] = preds

In [ ]:
SUB_FILE_NAME = f'mft_{ver}.csv'
ss.to_csv(SUB_FILE_NAME, index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>